In [1]:
import CalculatedFieldSubroutines as cfs

#

import numpy as np

import pandas as pd

#

import matplotlib.pyplot as plt

from pandasgui import show

#

import warnings

In [2]:
warnings.filterwarnings( 'ignore' )

In [3]:
gmID_list = cfs.list_whitelisted_gmIDs()

In [4]:
topic_list = cfs.list_topics()

print( topic_list )

['/apollo/canbus/chassis', '/apollo/drive/event', '/apollo/sensor/gnss/best/pose', '/apollo/perception/traffic/light']


In [38]:
def MergeChassisDriveEvent( chassis_df, drive_event_df ):

    chassis_time_array = np.array( chassis_df[ 'time' ] )

    #

    drive_event_time_array = np.array( drive_event_df[ 'time' ] )

    drive_event_event_array = np.array( drive_event_df[ 'event' ] )

    drive_event_type_array = np.array( drive_event_df[ 'type' ] )

    #

    chassis_num_rows = len( chassis_time_array )

    chassis_event_array = np.array( [ None for index in range( chassis_num_rows ) ] )

    chassis_type_array = np.array( [ None for index in range( chassis_num_rows ) ] )

    #

    for drive_event_time, drive_event_event, drive_event_type in zip( drive_event_time_array, drive_event_event_array,
                                                                      drive_event_type_array ):

        abs_time_diff_array = np.abs( chassis_time_array - drive_event_time )

        min_abs_time_diff_array_index = np.where( abs_time_diff_array == np.min( abs_time_diff_array ) )

        chassis_event_array[ min_abs_time_diff_array_index ] = drive_event_event

        chassis_type_array[ min_abs_time_diff_array_index ] = drive_event_type

    chassis_df[ 'DriveEvent' ] = chassis_event_array

    chassis_df[ 'DriveEventType' ] = chassis_type_array

In [39]:
def AssociateChassisDriveEvent( chassis_df, drive_event_df ):

    chassis_time_array = np.array( chassis_df[ 'time' ] )

    chassis_disengagement_time_array = np.array( chassis_df[ chassis_df[ 'TernaryDrivingModeTransition' ] == -1 ][ 'time' ] )

    #

    drive_event_time_array = np.array( drive_event_df[ 'time' ] )

    drive_event_event_array = np.array( drive_event_df[ 'event' ] )

    drive_event_type_array = np.array( drive_event_df[ 'type' ] )

    #

    chassis_num_rows = len( chassis_time_array )

    chassis_event_array = np.array( [ None for index in range( chassis_num_rows ) ] )

    chassis_type_array = np.array( [ None for index in range( chassis_num_rows ) ] )

    chassis_disengagement_time_diff_array = np.array( [ np.nan for index in range( chassis_num_rows ) ] )

    #

    if ( ( len( chassis_disengagement_time_array ) == 0 ) or ( len( drive_event_time_array ) == 0 ) ):

        chassis_df[ 'DriveEvent' ] = chassis_event_array

        chassis_df[ 'DriveEventType' ] = chassis_type_array

        chassis_df[ 'DriveEventTimeToDisengagement' ] = chassis_disengagement_time_diff_array 

        return None

    result = False

    for drive_event_time, drive_event_event, drive_event_type in zip( drive_event_time_array, drive_event_event_array, \
                                                                      drive_event_type_array ):

        disengagement_time_diff_array = chassis_disengagement_time_array - drive_event_time

        abs_disengagement_time_diff_array = np.abs( disengagement_time_diff_array )

        min_abs_disengagement_time_diff_array_index = np.where( abs_disengagement_time_diff_array == \
                                                                np.min( abs_disengagement_time_diff_array ) )

        chassis_disengagement_time = chassis_disengagement_time_array[ min_abs_disengagement_time_diff_array_index ]

        min_abs_disengagement_time_diff_array_index_conv = np.where( chassis_time_array == chassis_disengagement_time )

        if ( np.isnan( chassis_disengagement_time_diff_array[ min_abs_disengagement_time_diff_array_index_conv ] ) ):

            chassis_event_array[ min_abs_disengagement_time_diff_array_index_conv ] = drive_event_event

            chassis_type_array[ min_abs_disengagement_time_diff_array_index_conv ] = drive_event_type

            chassis_disengagement_time_diff_array[ min_abs_disengagement_time_diff_array_index_conv ] = \
                         disengagement_time_diff_array[ min_abs_disengagement_time_diff_array_index ]

        else:

            if ( np.abs( chassis_disengagement_time_diff_array[ min_abs_disengagement_time_diff_array_index_conv ] ) > \
                 np.abs( disengagement_time_diff_array[ min_abs_disengagement_time_diff_array_index ] ) ):

                chassis_event_array[ min_abs_disengagement_time_diff_array_index_conv ] = drive_event_event

                chassis_type_array[ min_abs_disengagement_time_diff_array_index_conv ] = drive_event_type

                chassis_disengagement_time_diff_array[ min_abs_disengagement_time_diff_array_index_conv ] = \
                             disengagement_time_diff_array[ min_abs_disengagement_time_diff_array_index ]

                result = True

    chassis_df[ 'DriveEvent' ] = chassis_event_array

    chassis_df[ 'DriveEventType' ] = chassis_type_array

    chassis_df[ 'DriveEventTimeToDisengagement' ] = chassis_disengagement_time_diff_array * 1e-9

    return result

In [41]:
merged_chassis_df_list = []

for index, gmID in enumerate( gmID_list ):

    temp_gmID_chassis_df = cfs.retrieve_gmID_topic( gmID_list[ index ], '/apollo/canbus/chassis' )

    temp_gmID_drive_event_df = cfs.retrieve_gmID_topic( gmID_list[ index ], '/apollo/drive/event' )

    temp_gmID_chassis_df.sort_values( 'time' )
    
    cfs.BinaryDrivingMode( temp_gmID_chassis_df )

    cfs.TernaryDrivingModeTransition( temp_gmID_chassis_df )

    MergeChassisDriveEvent( temp_gmID_chassis_df, temp_gmID_drive_event_df )

    merged_chassis_df_list.append( temp_gmID_chassis_df )

    print( index )

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246


In [42]:
show( merged_chassis_df_list[ 200 ] )

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
